## This is now somewhat outdated since our main support is redis now.
Still you can build on those building blocks they are still there.
Is not a lot work to make everything work with SQL.
Maybe we have moved them.

### We define a table schema which inherits most of the functionality from the database.

- auto generation of tables
- easy change the database

- since SQLite is so forgiving, pandas will check for correct dtypes and fail

### Known issues
- some prices display wrong in SQLite viewer, after querying they are fine
- the datatypes change, i guess that's how SQLite works

### Some special things about SQLite
All string datatypes in SQLite are converted to a TEXT datatype.  
If you try to specify a size for a string datatype, SQLite will ignore it.  
So long we have no performance loss we define them for convenience.

good fit because we can have one writer and many readers at the same time

In [ ]:
from mlrepricer import schemas, helper
from mlrepricer.database import SQLite
import pandas as pd

### The best way to work with a table is to instantiate the fitting tableobject.

In [ ]:
t = schemas.pricemonitor(SQLite)()  # pricemonitor table specific. SQLite is the destination database
# for a different database just change the destination
# here you don't have to change anything, you can just use the tableobjects attributes
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

The tableobject:

In [ ]:
tableobject = t
print(dir(tableobject))

In [ ]:
tableobject.createtable?

In [ ]:
tableobject.conn?

### Example how you can push data from a msg pack file into the database

In [ ]:
df_from_msgpack = helper.load_dataframe('alldata')

In [ ]:
# If you are not sure the database exists. You can always run this
tableobject.createtable

In [ ]:
df_from_msgpack.to_sql(tableobject.table, tableobject.conn,
                  dtype=tableobject.dtypes,
                  if_exists='append', index=False)

### How the schema looks after tableobject.create
CREATE TABLE price_monitor (  
	"ID" INTEGER NOT NULL,  
	asin NVARCHAR(40) NOT NULL,  
	feedback INTEGER NOT NULL,  
	feedbackpercent INTEGER NOT NULL,  
	instock BOOLEAN NOT NULL,  
	isbuyboxwinner BOOLEAN NOT NULL,  
	isfeaturedmerchant BOOLEAN NOT NULL,  
	isprime BOOLEAN NOT NULL,  
	price DECIMAL(10, 2) NOT NULL,  
	sellerid NVARCHAR(40) NOT NULL,  
	shipping_maxhours INTEGER NOT NULL,  
	shipping_minhours INTEGER NOT NULL,  
	time_changed DATETIME NOT NULL,  
	PRIMARY KEY ("ID"),  
	CHECK (instock IN (0, 1)),  
	CHECK (isbuyboxwinner IN (0, 1)),  
	CHECK (isfeaturedmerchant IN (0, 1)),  
	CHECK (isprime IN (0, 1))  
);